In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Massive connected components

Connected components crashes on 90m probabilities. We need to be able to handle that and more.

In [ ]:
import cmf
from cmf import clean
from cmf.dedupers import NaiveDeduper
from cmf.helpers import cleaner, cleaners, selector

from dwutils import s3

## Get and save dataframe

In [ ]:
_NAME = "naive_hmrc_exports_v1"
_SOURCE = "hmrc.trade__exporters"

exp_selector = selector(
    table=_SOURCE,
    fields=["company_name", "postcode"],
)

exp_raw = cmf.query(selector=exp_selector, return_type="pandas", limit=limit)

clean_exp = cleaners(
    cleaner(clean.company_name, {"column": "hmrc_trade__exporters_company_name"}),
    cleaner(clean.postcode, {"column": "hmrc_trade__exporters_postcode"}),
)

exp_clean = cmf.process(raw, exp_raw)

exp_naive_deduper = cmf.make_deduper(
    dedupe_run_name=_NAME,
    description="Basic cleaning of name and postcode.",
    deduper=NaiveDeduper,
    deduper_settings={
        "id": "data_sha1",
        "unique_fields": [
            "hmrc_trade__exporters_company_name",
            "hmrc_trade__exporters_postcode",
        ],
    },
    data=clean,
    data_source=_SOURCE,
)

exp_deduped = exp_naive_deduper()

with s3.write(path="hmrc_exporters_probabilities.parquet") as f:
    exp_deduped.dataframe.to_parquet(f)
